In [1]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from sklearn.metrics import r2_score
from sklearn import preprocessing

from sklearn.model_selection import KFold

from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import classification_report

import re

import warnings
warnings.filterwarnings('ignore')

In [2]:
df2 = pd.DataFrame(columns=['text', 'label'])
df = pd.read_csv('/Users/mac/Downloads/rusentitweet_train.csv')
for i, row in df.iterrows():
    if row[1] in ("negative", "positive"):
        df2.loc[len(df2.index)] = row
df = df2

In [3]:
import nltk
from nltk.corpus import stopwords
 
nltk.download('stopwords')
stop = stopwords.words('russian')
# Стоп-слова без не
stop.remove('не')

[nltk_data] Downloading package stopwords to /Users/mac/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:

s=[]
s1 = ""
regs = [r'https?:\/\/.\S+', r'#\S+', r'@\S+', r'\\r\\n', r'[-.)(,:]']
glasn = ['о', 'о', 'а', 'я', 'а', 'а', 'и', 'а', 'е', 'у', 'о', 'о', 'а', 'е', 'я', 'о', 'у']
signs = ['!', '_', '.']

# приведение всего к нижнему регистру
for i, row in df.iterrows():
    row[0] = row[0].lower()

df1 = df    

for i, row in df1.iterrows():
    # удаление ссыло, хэштегов(и знака решетки и последующего хэштега, потому что слова после хэштэга пишутся слитно)
    # удаление отмеченных пользователей(@name), удаление знаков пунктуации (кроме ! ?)
    for reg in regs:
        row[0] = re.sub(reg, "", row[0])
        
    # удаление стоп-слов
    row[0] = ' '.join([x for x in row[0].split() if x not in stop])
    
    # удаление повторящихся подряд смайликов и знаков препинания (оставшихся)
    # удаление подряд идущих гласных (тыыыыы, яяяяяя, аааааа и тд)
    # удаление цифр, так как по сути цифры не имеют никакого значения в данном анализе
    r = row[0]
    for t in range(0, len(row[0])):
        if not(not(r[t].isalpha()) and r[t] == r[t-1]) and not((r[t] in glasn) and r[t] == r[t-1]) and not(r[t].isdigit()):
            s1+=r[t]
    row[0] = s1
    
    s1 = ""
    
    r = row[0] + "  "
    
    #отделение смайлика от слова (чимина🤧 -> чимина 🤧)
    for t in range(0, len(row[0])):
        if not(r[t+1].isalpha()) and r[t]!= " " and not(r[t+1].isdigit()) and (r[t+2].isalpha() or r[t+2]==" "):
            s1+=r[t] + " "
        else:
            s1+=r[t]
    row[0] = s1
    
    s = []
    s1=""
df1

,text,label
0,помойму вкрашилась чимина 🤧,positive
1,не порядке !,negative
2,следующий буду победного закрывать пожелай...,positive
3,удивительный гiмн удивительно пока ещё не...,negative
4,срала биологию,negative
...,...,...
4564,дедрадио хочешь сказать лайкаешь не мои ...,negative
4565,действительно любовь такая,positive
4566,хоби утро добрым 😂,positive
4567,не успела встать заебалась нового,negative


In [5]:
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
from functools import reduce

# стемминг
new = SnowballStemmer(language='russian')
for i, row in df1.iterrows():
    row[0] = ' '.join([new.stem(x) for x in row[0].split()])
df

,text,label
0,помойм вкраш чимин 🤧,positive
1,не порядк !,negative
2,след буд победн закрыва пожела удач !,positive
3,удивительн гимн удивительн пок ещ не сдохл украин,negative
4,срал биолог,negative
...,...,...
4564,дедрад хочеш сказа лайка не мо твиты? 😡 пздц у...,negative
4565,действительн любов так,positive
4566,хоб утр добр 😂,positive
4567,не успел встат заеба нов,negative


In [6]:
arr = []

for i in range(0, len(df1)):
    arr.append([])

for i, row in df1.iterrows():
    for t in row[0].split():
        arr[i].append(t)
# arr
b = []
for i, row in df1.iterrows():
    b.append(row[0])
b

['помойм вкраш чимин 🤧',
 'не порядк !',
 'след буд победн закрыва пожела удач !',
 'удивительн гимн удивительн пок ещ не сдохл украин',
 'срал биолог',
 'помим алин ещ радост гемоглобин повыс',
 'пиздец че вобщ чувств жизн помота прошл сутк туман',
 'спичк ? зажеч огон тво глаз',
 'эт сам дел очен крут',
 'хоч сказа чтот приятн получа "ид нах ебла противн "',
 'сук огреба',
 'темн прошл патриарх кирилл владык стал миллиардер — инсайд',
 'проигра укулел кукушк ком интересн очен прост аккорд темп вполн легк крут',
 'лезбиянк эт твит 🤔',
 'классспасиб доставк galaxy buds bts украинупоиск папик отмен',
 'бля паладин неубива',
 'отц сдела ментальн инвалид',
 'эт точно! 👍',
 'пок имен любл',
 'й красив числ',
 'заплат сем минут потрат узна основн тип рифмовк',
 'соболезн',
 'закр вскро',
 'нрав професс содержанк посовет вуз сответств программ бакалавриат / магистратур',
 'пиздец какойт поход осозна сон слов',
 'больш сем не крыс',
 'разыгрыва 🤣 написа валентинк препод физик подлож лиц неопи

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer


vec = TfidfVectorizer()
matr = vec.fit_transform(b)
# tf-idf
print(matr)
print(vec.get_feature_names_out())

vect = CountVectorizer()
m = vect.fit_transform(b)
fm = vect.get_feature_names_out()
# count matrix
print(m.toarray())
print(m.toarray().shape)


  (0, 7569)	0.5346549574878128
  (0, 822)	0.5831906314872191
  (0, 4986)	0.6115821807240984
  (1, 5069)	0.9453398497311548
  (1, 3661)	0.32608674997656334
  (2, 7003)	0.39675554325904605
  (2, 4854)	0.4497167039101802
  (2, 2035)	0.43418293347176135
  (2, 4699)	0.47161032368447664
  (2, 603)	0.2853237257897314
  (2, 6167)	0.3847066148312607
  (3, 7047)	0.32578380490864045
  (3, 6005)	0.3538758157392728
  (3, 1842)	0.23047534074055107
  (3, 4893)	0.26595176497621503
  (3, 1213)	0.3860004617159947
  (3, 7009)	0.6934630758718668
  (3, 3661)	0.12206638139981257
  (4, 456)	0.6843936599995228
  (4, 6441)	0.7291126923545203
  (5, 4726)	0.4568743996857909
  (5, 1199)	0.4568743996857909
  (5, 5623)	0.372686081875884
  (5, 198)	0.4089440168202436
  (5, 4975)	0.4568743996857909
  :	:
  (4564, 7000)	0.34741235178200414
  (4564, 4562)	0.34741235178200414
  (4564, 7435)	0.32323400796287866
  (4564, 2936)	0.3598417295949482
  (4564, 3351)	0.1983547539566139
  (4564, 6109)	0.24096093983983433
  (4564,

In [9]:
count = []
print(len(df1))
k=0
for i, row in df1.iterrows():
    for t in row[0].split():
        arr = []
        for j, rowj in df1.iterrows():
            arr.append(rowj[0].count(t))
        count.append([])
        count[k].append(arr)
        k+=1
count
print(1)

4569
1
